Já fiz o uso de arquivos deltas no databricks, porém não consegui importar
no colab. Tentei por 30 minutos, ocorreram erros, decidi não focar nisso devido ao tempo que tenho para dedicar.

Farei como se fossem vários parquets numa pasta e quisesse ler todos


O dataset possui as seguintes colunas: created_at (unix timestamp de quando a requisição foi recebida), publisher (string, uuid identificando o local onde o usuário estava navegando), event_type (string, identifica o tipo requisição), ip (string, semi anonimizado), user_id (string, uuid identificando o usuário), geo_lat (double, latitude onde o usuário estava fisicamente), geo_lon (double, longiture onde o usuário estava fisicamente), geo_country (string, país da localização latitude-longitude), geo_region (string, estado da localização latitude-longitude), geo_city (string, cidade da localização latitude-longitude).

In [ ]:
import pandas as pd
import os
import geopandas as gpd
import plotly.express as px
from glob import glob
from shapely.geometry import Point


In [ ]:
# bases descontos
df_paths = glob('/content/drive/MyDrive/Caze_zedia/format_1/*')

df_list = [pd.read_parquet(table)
               for table in df_paths]

# concatenando historico
df = pd.concat(df_list).reset_index(drop=True)

In [ ]:
# Bases IBGE de informações de renda
df_ibge = pd.read_excel(
    '/content/drive/MyDrive/Caze_zedia/ResponsavelRenda_PR.xls',
     usecols=['Cod_setor', 'Situacao_setor', 'V001', 'V002', 'V003',
         'V004', 'V005', 'V006', 'V007', 'V008', 'V009', 'V021'])

# Lendo arquivos shapefile do ibge
setores = gpd.read_file("/content/drive/MyDrive/Caze_zedia/41SEE250GC_SIR.shp")


In [ ]:
# Entendendo se as colunas vieram corretamente (nome e formato)
df.info()

In [ ]:
# Amostra de dados para ficar no radar

df.sample(10)

In [ ]:
# Entendendo a quantidades de labels por coluna para possíveis agrupamentos
df.nunique()

In [ ]:
# Verificando distribuições dos tipos de requisição no server
df['event_type'].value_counts()

In [ ]:
# Verificando distribuições dos tipos de requisição no server
df['event_type'].value_counts()

# Visualização da diferença muito alta de frequência
fig = px.histogram(df,
                   x="event_type",
                   title='Distribuição de frequência de requisições')
fig.update_layout(
        yaxis_title="frequência",
        xaxis_title="Requisição",
        template="simple_white")

fig.show()

In [ ]:
# Verificando a distribuição de frequências por estados
# Algumas letras e número no meio dos estados - 11, 10 e N
# Interessante aderencia no PR e SC, poém não no RS

frequency_df = df['geo_region'].value_counts().reset_index()
frequency_df.columns = ['geo_region', 'frequency']

# Ordenar pelo valor de frequência - Visto no GPT como fazer
frequency_df = frequency_df.sort_values(by='frequency', ascending=False)

# Criar o histograma com as categorias ordenadas
fig = px.histogram(
    frequency_df,
    x='geo_region',
    y='frequency',
    title='Distribuição de frequência de geo regions',
    category_orders={'geo_region': frequency_df['geo_region']}
)

# Atualizar o layout do gráfico
fig.update_layout(
    yaxis_title='Frequência',
    xaxis_title='Requisição',
    template='simple_white'
)

# Exibir o gráfico
fig.show()


In [ ]:
# Agrupando por data e quantidade de requisicoes com contagem
df_grouped = df.groupby(by=['created_at', 'event_type'])['event_type']\
  .count().reset_index(name='count')

# Verificando séries temporais das quantidades de requisições recebidas
df_grouped = df_grouped.sort_values(by='created_at')

for requisicao in df_grouped['event_type'].unique():

    df_plot = df_grouped.loc[df_grouped['event_type'] == requisicao]

    # Serie temporaç
    fig = px.line(
        df_plot,
        x='created_at', y='count',
        color='event_type',
        title=f'Quantidade recebidas | Requisicao = {requisicao}')

    fig.update_layout(
        yaxis_title="Qtd de requisições",
        xaxis_title="Tempo",
        template="simple_white")

    fig.show()

In [ ]:
# As séries não parecem orgânicas, estão estranhas então irei conferir
# A série poll-answer foi escolhida pois é uma constante, logo fácil de
# conferir. As séreis foram plotadas corretamente


df.loc[df['event_type'] == 'poll-answer']

In [ ]:
# Entendendo se o somatório das requisições segue um padrão mais orgânico
# É esperado que reproduza um sinal parecido com as dos primeiros sinais da
# célula anterior - Devido a freuência muito maior
# Agrupando por data e contando requisições
df_grouped = df.groupby(by=['created_at'])['event_type']\
  .count().reset_index()

# Verificando séries temporais das quantidades de requisições recebidas
df_grouped = df_grouped.sort_values(by='created_at')

# Serie temporal
fig = px.line(
    df_grouped,
    x='created_at', y='event_type',
    title=f'Quantidade de requisições recebidas')

fig.update_layout(
    yaxis_title="Qtd de requisições",
    xaxis_title="Tempo",
    template="simple_white")

fig.show()

In [ ]:
# Os resultados mostraram que existem geo regiões que não são estados
# Recurrent-ping escolhido pela maior frequência

# Os estados que possuem relevância no número e requisições
# seguem o mesmo padrão já visto no agregado do tipo de requisição

# Também entendendo se existia algum estado que poderia ser
# um servidor de teste sujando a sérei, pois as vzes apenas
# uma série está sujando a série e o restande tem uma
# possibilidade maior de previsão


df_recurrent_ping = df.loc[df['event_type'] == 'recurrent-ping']

# Gerado pelo GPT - não lembrava como agrupar dois eventos
df_grouped = (
    df_recurrent_ping.groupby(
        ['created_at', 'event_type', 'geo_region']
    ).size()  # Usando .size() para contar as aparições
    .reset_index(name='count')  # Renomeando a coluna de contagem
)

# Verificando séries temporais das quantidades de requisições recebidas
df_grouped = df_grouped.sort_values(by='created_at')

for estado in df_grouped['geo_region'].unique():

    df_plot = df_grouped.loc[df_grouped['geo_region'] == estado]

    # Serie temporaç
    fig = px.line(
        df_plot,
        x='created_at', y='count',
        color='event_type',
        title=f'Quantidade recebidas | Estado = {estado}')

    fig.update_layout(
        yaxis_title="Qtd de requisições",
        xaxis_title="Tempo",
        template="simple_white")

    fig.show()

In [ ]:
# Verificando as séries dos locais de navegação
# recurrent-ping escolhido pela maior frequência
# Entendendo se algum publisher pode estar sujando

df_recurrent_ping = df.loc[df['event_type'] == 'recurrent-ping']

# Gerado pelo GPT - não lembrava como agrupar dois eventos
df_grouped = (
    df_recurrent_ping.groupby(
        ['created_at', 'event_type', 'publisher']
    ).size()  # Usando .size() para contar as aparições
    .reset_index(name='count')  # Renomeando a coluna de contagem
)

# Verificando séries temporais das quantidades de requisições recebidas
df_grouped = df_grouped.sort_values(by='created_at')

for publisher in df_grouped['publisher'].unique():

    df_plot = df_grouped.loc[df_grouped['publisher'] == publisher]

    # Serie temporaç
    fig = px.line(
        df_plot,
        x='created_at', y='count',
        color='event_type',
        title=f'Quantidade recebidas | publisher = {publisher}')

    fig.update_layout(
        yaxis_title="Qtd de requisições",
        xaxis_title="Tempo",
        template="simple_white")

    fig.show()

In [ ]:
# Apenas um estado gerou a requisição poll-answer

df_poll_answer = df.loc[df['event_type'] == 'poll-answer']

# Gerado pelo GPT - não lembrava como agrupar dois eventos
df_grouped = (
    df_poll_answer.groupby(
        ['created_at', 'event_type', 'geo_region']
    ).size()  # Usando .size() para contar as aparições
    .reset_index(name='count')  # Renomeando a coluna de contagem
)

# Verificando séries temporais das quantidades de requisições recebidas
df_grouped = df_grouped.sort_values(by='created_at')

for estado in df_grouped['geo_region'].unique():

    df_plot = df_grouped.loc[df_grouped['geo_region'] == estado]

    # Serie temporaç
    fig = px.line(
        df_plot,
        x='created_at', y='count',
        color='event_type',
        title=f'Quantidade recebidas | Estado = {estado}')

    fig.update_layout(
        yaxis_title="Qtd de requisições",
        xaxis_title="Tempo",
        template="simple_white")

    fig.show()

Ontem o trabalho seguiu mais reflexivo de entendimento das bases, em um momento
menos "produtivo" e mais criativo.

Devido as séries com aparência não orgânica e baixo entendimento do negócio nos
picos de requisições surgiram ideias, porém com baixa aplicabilidade real.
Das ideias que surgiram: é possível entender anomalias ou em caso de
picos com causas conhecidos e desejados, verificar uplift dessas causas com a ideia de prever demandas futuras para preparo de servidores e afins.

Porém, para ser mais realista, algo que vai ser útil idependentemente dos tipos de dados é o enriquecimento com dados de renda dos setores censitários das requisições, visto que existe o produto "anuncio" e ele deve levar caracteristica de renda na sua distribuição.


In [ ]:
# Dicionário para identificação -  Retirado da documentação do ibge
dicionario = {'V001': 'meio salario', 'V002': 'meio a 1 salario',
              'V003': '1 a 2 salarios', 'V004': '2 a 3 salarios',
              'V005': '3 a 5 salarios', 'V006': '5 a 10 salarios',
              'V007': '10 a 15 salarios', 'V008': '15 a 20 salarios',
              'V009': 'maior que 20 salarios', 'V021': 'total_responsaveis',
              'Cod_setor': 'setor_censitario'}

df_ibge = df_ibge.rename(columns=dicionario)

result = df_ibge.dropna().drop_duplicates()
result = result.reset_index(drop=True)

# convertendo para numérico
result[['meio salario', 'meio a 1 salario', '1 a 2 salarios',
        '2 a 3 salarios', '3 a 5 salarios', '5 a 10 salarios',
        '10 a 15 salarios', '15 a 20 salarios', 'maior que 20 salarios',
        'total_responsaveis']] =\
        pd.to_numeric(
        result[['meio salario', 'meio a 1 salario', '1 a 2 salarios',
                '2 a 3 salarios', '3 a 5 salarios', '5 a 10 salarios',
                '10 a 15 salarios', '15 a 20 salarios',
                'maior que 20 salarios',
                'total_responsaveis']].stack(), errors='coerce').unstack()

# Normaliza para permitir comparações da participação da faixa de renda no
# no total, e poder somar para chegar na média de salários
result['meio salario'] =\
    result['meio salario'] * 0.5 / result['total_responsaveis']
result['meio a 1 salario'] =\
    result['meio a 1 salario'] * 0.75 / result['total_responsaveis']
result['1 a 2 salarios'] =\
    result['1 a 2 salarios'] * 1.5 / result['total_responsaveis']
result['2 a 3 salarios'] =\
    result['2 a 3 salarios'] * 2.5 / result['total_responsaveis']
result['3 a 5 salarios'] =\
    result['3 a 5 salarios'] * 3.5 / result['total_responsaveis']
result['5 a 10 salarios'] =\
    result['5 a 10 salarios'] * 7.5 / result['total_responsaveis']
result['10 a 15 salarios'] =\
    result['10 a 15 salarios'] * 12.5 / result['total_responsaveis']
result['15 a 20 salarios'] =\
    result['15 a 20 salarios'] * 17.5 / result['total_responsaveis']
result['maior que 20 salarios'] =\
    result['maior que 20 salarios'] * 20 / result['total_responsaveis']

# Para calcular a renda média
result['renda_media'] = (result['meio salario'] + result['meio a 1 salario'] +
                         result['1 a 2 salarios'] + result['2 a 3 salarios'] +
                         result['3 a 5 salarios'] + result['5 a 10 salarios'] +
                         result['10 a 15 salarios'] + result['15 a 20 salarios'] +
                         result['maior que 20 salarios'])




In [ ]:
setores.info()

In [ ]:
# Unindo tabelas IBGE
result['setor_censitario'] = result['setor_censitario'].astype(str)
df_ibge_merged = result.merge(
    setores, left_on='setor_censitario', right_on='CD_GEOCODI', how='left')

# Deixando apenas paraná, foi escolhido pois é o estado onde existe o maior
# número de requisições

df_pr = df.loc[df['geo_region'] == 'PR'].copy()

In [ ]:
# Adaptado do GPT de outro código meu
# Converte latitude e longitude para objetos de ponto
df_pr['geometry'] = df_pr.apply(
    lambda row: Point(row['geo_lon'], row['geo_lat']), axis=1)

# Converte para um GeoDataFrame
# com o mesmo sistema de coordenadas do setor censitário
gdf_coords = gpd.GeoDataFrame(df_pr, geometry='geometry', crs="EPSG:4674")

In [ ]:
# Realiza a junção espacial
merged_gdf = gpd.sjoin(gdf_coords, setores, how="inner", predicate="within")

In [ ]:
merged_gdf.info()

Pronto agora temos mais informações e podemos clusterizar, existe outras maneiras de enriquecer, como quanitdade de homens e mulheres para segmentação de anúncios, entre outras possibilidades.